In [1]:
import pandas as pd

import DataRetriever as dr
import SARIMA_forecast
import FFT_forecast

RETRIEVER = dr.DataRetriever()
CON_ATTRIBUTES = RETRIEVER.get_attributes(file_name='consuming_attributes.pkl')
PV_ATTRIBUTES = RETRIEVER.get_attributes(file_name='producing_attributes.pkl')
FLEX_ATTRIBUTES = ["Load_ClothesWasherPowerWithStandby", "Elec_PowerDishwasher", "Load_DryerPowerTotal"]
FIXED_ATTRIBUTES = list(set(CON_ATTRIBUTES) - set(FLEX_ATTRIBUTES))

In [2]:
def rescheduler(year, month, day):
    date = pd.Timestamp(year=year, month=month, day=day)
    pv = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[PV_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000
    fixed = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[FIXED_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000

    flex_clothes = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_ClothesWasherPowerWithStandby"].clip(lower=0) / 1000
    flex_dish = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Elec_PowerDishwasher"].clip(lower=0) / 1000
    flex_dryer = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_DryerPowerTotal"].clip(lower=0) / 1000

    # 7 days of training data - subject to be changed, depends on fcast model
    pv = pv.loc[(pv.index >= date - pd.Timedelta(days=7)) & (pv.index < date)]
    # fixed = fixed.loc[(fixed.index >= date - pd.Timedelta(days=7)) & (fixed.index < date)]

    flex_clothes = flex_clothes.loc[(flex_clothes.index >= date) & (flex_clothes.index < date + pd.Timedelta(days=3))]
    flex_dish = flex_dish.loc[(flex_dish.index >= date) & (flex_dish.index < date + pd.Timedelta(days=3))]
    flex_dryer = flex_dryer.loc[(flex_dryer.index >= date) & (flex_dryer.index < date + pd.Timedelta(days=3))]

    # ------------------ Forecast PV ------------------
    pv_pred = FFT_forecast.fourierExtrapolation(data=pv, number_of_predictions=72, n_sinusoids=5)[len(pv):]
    print("PV Prediction Finished")

    # ------------------ Forecast Fixed ------------------
    fixed_pred = SARIMA_forecast.sarima_prediction(date, fixed)
    print("Fixed Prediction Finished")

    # Concat results
    df = pd.concat([pd.DataFrame(pv_pred).reindex(flex_dish.index), fixed_pred.reindex(flex_dish.index), flex_clothes, flex_dish, flex_dryer], axis=1)
    # Easy way to remove column names
    df = df.T.reset_index(drop=True).T
    df.rename(columns={0: "PV", 1: "FIXED", 2: "FLEX_CLOTHES", 3: "FLEX_DISH", 4: "FLEX_DRYER"}, inplace=True)

    # Stand-by values set to 0, simplifies the code
    df.loc[df["FLEX_CLOTHES"] <= 0.002947, "FLEX_CLOTHES"] = 0

    df["Balance"] = df["PV"] - (df["FIXED"] + df["FLEX_CLOTHES"] + df["FLEX_DISH"] + df["FLEX_DRYER"])

    # pd.DataFrames with flexible appliances turned on and surplus where at least one flexible is not turned on.
    deficit = df[(df["Balance"] < 0) & ((df["FLEX_DISH"] > 0) | (df["FLEX_DRYER"] > 0) | (df["FLEX_CLOTHES"] > 0))]
    surplus = df[(df["Balance"] > 0) & ((df["FLEX_DISH"] == 0) | (df["FLEX_DRYER"] == 0) | (df["FLEX_CLOTHES"] == 0))]

    columns_dict = {0: "FLEX_CLOTHES", 1: "FLEX_DISH", 2: "FLEX_DRYER"}
    res_df = pd.DataFrame(columns=["Planned Time", "Appliance", "Suggested Time"])

    # Loops over all rows in the deficit to find better times to turn on appliances.
    for idx, row in deficit.iterrows():
        while row[["FLEX_CLOTHES", "FLEX_DISH", "FLEX_DRYER"]].max() != 0: # Thus, a flexible appliance is active.
            col = columns_dict[row[["FLEX_CLOTHES", "FLEX_DISH", "FLEX_DRYER"]].argmax()] # Column of the largest deficit.

            deficit_value = deficit.at[idx, col] # Significance of deficit
            surplus_possible = surplus[(surplus["Balance"] >= deficit_value) & (surplus[col] == 0)] # Possible time slots

            suggested_time = surplus_possible.index[surplus_possible.index.get_indexer([idx], method='nearest')][0] # Closest time with enough surplus

            surplus.at[suggested_time, col] = deficit_value
            surplus.at[suggested_time, "Balance"] -= deficit_value # Change balance in the surplus df

            deficit.at[idx, col] = 0 # Set the deficit value to 0

            res_df.loc[len(res_df)] = (idx, col, suggested_time) # Insert rows to final result

    return res_df

In [ ]:
suggested_rescheduler = rescheduler(2016, 1, 21)
suggested_rescheduler

PV Prediction Finished


In [ ]:
date = pd.Timestamp(year=2016, month=1, day=21)
pv = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[PV_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000
fixed = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[FIXED_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000

flex_clothes = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_ClothesWasherPowerWithStandby"].clip(lower=0) / 1000
flex_dish = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Elec_PowerDishwasher"].clip(lower=0) / 1000
flex_dryer = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_DryerPowerTotal"].clip(lower=0) / 1000

pv = pv.loc[(pv.index >= date) & (pv.index < date + pd.Timedelta(days=3))]
fixed = fixed.loc[(fixed.index >= date) & (fixed.index < date + pd.Timedelta(days=3))]

flex_clothes = flex_clothes.loc[(flex_clothes.index >= date) & (flex_clothes.index < date + pd.Timedelta(days=3))]
flex_dish = flex_dish.loc[(flex_dish.index >= date) & (flex_dish.index < date + pd.Timedelta(days=3))]
flex_dryer = flex_dryer.loc[(flex_dryer.index >= date) & (flex_dryer.index < date + pd.Timedelta(days=3))]


df = pd.concat([pv.reindex(flex_dish.index), fixed.reindex(flex_dish.index), flex_clothes, flex_dish, flex_dryer], axis=1)
# Easy way to remove column names
df = df.T.reset_index(drop=True).T
df.rename(columns={0: "PV", 1: "FIXED", 2: "FLEX_CLOTHES", 3: "FLEX_DISH", 4: "FLEX_DRYER"}, inplace=True)

# Stand-by values set to 0, simplifies the code
df.loc[df["FLEX_CLOTHES"] <= 0.002947, "FLEX_CLOTHES"] = 0

df["Balance"] = df["PV"] - (df["FIXED"] + df["FLEX_CLOTHES"] + df["FLEX_DISH"] + df["FLEX_DRYER"])
df

In [ ]:
balance_pre = sum(df["Balance"][df["Balance"] < 0])
balance_pre

In [ ]:
for idx, row in suggested_rescheduler.iterrows():
    df.at[row["Suggested Time"], row["Appliance"]] = df.at[row["Planned Time"], row["Appliance"]]
    df.at[row["Planned Time"], row["Appliance"]] = 0

In [ ]:
df["New_Balance"] = df["PV"] - (df["FIXED"] + df["FLEX_CLOTHES"] + df["FLEX_DISH"] + df["FLEX_DRYER"])
df

In [ ]:
balance_post = sum(df["New_Balance"][df["New_Balance"] < 0])
balance_post